In [ ]:
!pip install --upgrade diffusers transformers scipy ftfy
!pip install flax==0.5.0 --no-deps
!pip install ipywidgets msgpack rich 

In [ ]:
#Your need a Hugging Face account and token for this to work. Info on on how to get your token here: https://huggingface.co/docs/hub/security-tokens
#Once you have your token, replace 'YOUR_HF_TOKEN_GOES_HERE' below with your token

!wget https://raw.githubusercontent.com/gradient-ai/stable-diffusion/main/login.py
!python login.py --token YOUR_HF_TOKEN_GOES_HERE

In [ ]:
!mkdir outputs

# Inference

In order to generate an image, you simply need to run one of the two cells below. The first cell is optimized for low power GPUs, like the Free GPU M4000, and will be able to generate an image on any GPU powered Gradient Machine. 

The next cell, is optimized to run on more powerful GPU setups, like an A100 or A6000, and can be used to quickly generate high quality images on these machines. 

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "your-account/model-v1"
device = "cuda"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()

In [ ]:
sample_num = 3
lst = []
prompt = '''A photo of sks'''

for i in range(sample_num):
    with autocast("cuda"):
        a = pipe(prompt, guidance_scale=7.5, height=512, width=512,
                 num_inference_steps=100, seed=343434, scheduler='LMSDiscreteScheduler')["images"][0]
        lst.append(a)
        display(a)
        a.save(f'outputs/gen-image-{i}.png')

In [ ]:
#Clear memory
torch.cuda.empty_cache()
import gc
gc.collect()